In [1]:
import os
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset
from torch.utils.tensorboard import SummaryWriter

In [ ]:
num_workers = os.cpu_count()
print(f"Number of workers: {num_workers}")

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

#### Load data

In [7]:
bc = datasets.load_wine()
X, y = bc.data, bc.target

In [8]:
X = torch.from_numpy(X.astype(np.float32))
y = torch.from_numpy(y.astype(np.int64))
# y = y.view(y.shape[0], 1)
n_samples, n_features = X.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, test_size=0.2, random_state=1
)

X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, stratify=y_train, test_size=0.2, random_state=1
)

X_train.shape, X_val.shape, X_test.shape

#### Prepare dataset, dataloader

In [10]:
class LRDataset(Dataset):

    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __getitem__(self, index):
        return self.X[index], self.y[index]

    def __len__(self):
        return len(self.y)

In [11]:
train_ds = LRDataset(X_train, y_train)
val_ds = LRDataset(X_val, y_val)
test_ds = LRDataset(X_test, y_test)

In [12]:
num_workers = 0

train_dl = DataLoader(
    dataset=train_ds, batch_size=32, shuffle=True, num_workers=num_workers
)
val_dl = DataLoader(
    dataset=val_ds, batch_size=32, shuffle=False, num_workers=num_workers
)
test_dl = DataLoader(
    dataset=test_ds, batch_size=32, shuffle=False, num_workers=num_workers
)

#### Define Model

In [13]:
class LogisticRegression(nn.Module):

    def __init__(self, n_features, n_classes):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(n_features, n_classes)

    def forward(self, x):
        return self.linear(x)

#### Instantiate Model and Define Loss & Optimizer

In [14]:
model = LogisticRegression(n_features=n_features, n_classes=3).to(device)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.05)

#### Run training

In [15]:
writer = SummaryWriter(log_dir="../../runs/logistic_regression")

In [16]:
verbose = 5
n_epochs = 100
train_losses = []
val_losses = []

In [ ]:
for epoch in tqdm(range(n_epochs)):
    model.train()
    epoch_loss = 0
    for X_batch, y_batch in train_dl:
        # Move data to device
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        # Forward pass
        y_pred = model(X_batch)
        loss = loss_fn(y_pred, y_batch)

        # Backward pass
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        epoch_loss += loss.item()

    epoch_loss /= len(train_dl)
    train_losses.append(epoch_loss)

    # Validation loss calculation
    model.eval()
    with torch.no_grad():
        val_loss = 0
        for X_val_batch, y_val_batch in val_dl:
            X_val_batch, y_val_batch = X_val_batch.to(device), y_val_batch.to(device)
            y_val_pred = model(X_val_batch)
            val_loss += loss_fn(y_val_pred, y_val_batch).item()

        val_loss /= len(val_dl)
        val_losses.append(val_loss)

    writer.add_scalars("Loss", {"Train": epoch_loss, "Validation": val_loss}, epoch)
    if verbose and (epoch + 1) % verbose == 0:
        print(
            f"Epoch {epoch + 1}/{n_epochs} | Training Loss: {epoch_loss:.4f} | Validation Loss: {val_loss:.4f}"
        )

print(f"Final Training Loss: {epoch_loss:.4f} | Validation Loss: {val_loss:.4f}")

In [18]:
# Plot losses
writer.flush()
writer.close()